In [1]:
import numpy as np
import xarray as xr
import scipy.signal as sg
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
from matplotlib import patches
import npc_lims
# from sklearn.metrics import roc_curve, roc_auc_score
# from statsmodels.stats.multitest import fdrcorrection
# from npc_sessions import DynamicRoutingSession
# from dynamic_routing_analysis import spike_utils
import npc_ephys
import os

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
ephys_sessions=tuple(s for s in npc_lims.get_session_info(is_ephys=True, is_uploaded=True,))

c:\Anaconda3\envs\npc_analysis\Lib\site-packages\npc_lims\metadata\codeocean.py:207: UserWarning: There is more than one asset for session = '644866_2023-02-07'. Defaulting to most recent: ('ecephys_644866_2023-02-07_17-08-15', 'ecephys_644866_2023-02-07_17-08-15')
  warnings.warn(
c:\Anaconda3\envs\npc_analysis\Lib\site-packages\npc_lims\metadata\codeocean.py:207: UserWarning: There is more than one asset for session = '664851_2023-11-14'. Defaulting to most recent: ('ecephys_664851_2023-11-14_12-44-33', 'ecephys_664851_2023-11-14_12-44-33')
  warnings.warn(
c:\Anaconda3\envs\npc_analysis\Lib\site-packages\npc_lims\metadata\codeocean.py:207: UserWarning: There is more than one asset for session = '686176_2023-12-06'. Defaulting to most recent: ('ecephys_686176_2023-12-06_13-03-34', 'ecephys_686176_2023-12-06_13-03-34')
  warnings.warn(
c:\Anaconda3\envs\npc_analysis\Lib\site-packages\npc_lims\metadata\codeocean.py:207: UserWarning: There is more than one asset for session = '644866_20

In [18]:
# npc_lims.get_surface_channel_root(ephys_sessions[3])

In [19]:
#get data assets that were sorted in 2024
import npc_lims
import npc_session

latest_sorted_sessions = []
sessions_not_sorted = []
sorted_surface_channel_assets = []
sorted_surface_channel_assets_not_sorted = []

for session_info in npc_lims.get_session_info(is_ephys=True, is_uploaded=True):
    try:
        sorted_asset = npc_lims.get_session_sorted_data_asset(session_info.id)
    except Exception as e:
        print(e)
        continue
    if '2024' in sorted_asset['name']:
        latest_sorted_sessions.append(session_info.id)
    else:
        sessions_not_sorted.append(session_info.id)

    if session_info.is_surface_channels:
        try:
            sorted_surface_channel_asset = npc_lims.get_session_sorted_data_asset(npc_session.SessionRecord(session_info.id).with_idx(1))
            if '2024' in sorted_surface_channel_asset['name']:
                sorted_surface_channel_assets.append(session_info.id)
            else:
                sorted_surface_channel_assets_not_sorted.append(session_info.id)
        except (ValueError, FileNotFoundError):
            sorted_surface_channel_assets_not_sorted.append(session_info.id)

c:\Anaconda3\envs\npc_analysis\Lib\site-packages\npc_lims\metadata\codeocean.py:207: UserWarning: There is more than one asset for session = '702136_2024-03-07'. Defaulting to most recent: ('ecephys_702136_2024-03-07_12-50-28_sorted_2024-03-27_09-41-38', 'ecephys_702136_2024-03-07_14-49-27_sorted_2024-03-30_00-55-51', 'ecephys_702136_2024-03-07_12-50-28_sorted_2024-03-26_04-29-16')
  warnings.warn(
c:\Anaconda3\envs\npc_analysis\Lib\site-packages\npc_lims\metadata\codeocean.py:207: UserWarning: There is more than one asset for session = '702136_2024-03-06'. Defaulting to most recent: ('ecephys_702136_2024-03-06_11-25-22_sorted_2024-03-27_09-41-45', 'ecephys_702136_2024-03-06_11-25-22_sorted_2024-03-27_10-01-07')
  warnings.warn(
c:\Anaconda3\envs\npc_analysis\Lib\site-packages\npc_lims\metadata\codeocean.py:207: UserWarning: There is more than one asset for session = '676909_2023-12-14'. Defaulting to most recent: ('ecephys_676909_2023-12-14_12-43-11_sorted_2024-03-01_16-02-16', 'eceph

In [20]:
sessions_not_sorted

[]

In [21]:
sorted_surface_channel_assets_not_sorted

[]

In [22]:
sorted_surface_channel_assets

['702136_2024-03-07',
 '690706_2023-11-30',
 '690706_2023-11-29',
 '668755_2023-08-29',
 '668755_2023-08-28',
 '662892_2023-08-24',
 '662892_2023-08-23',
 '660023_2023-08-09',
 '660023_2023-08-08']

In [ ]:
sorted_surface_channel_asset = npc_lims.get_session_sorted_data_asset(npc_session.SessionRecord('660023_2023-08-08').with_idx(1))

In [ ]:
surface_channel_si = npc_ephys.SpikeInterfaceKS25Data(npc_session.SessionRecord('660023_2023-08-08').with_idx(1))

In [ ]:
waveforms_surface=surface_channel_si.get_nwb_units_device_property('waveform_mean','probe'+probe)

In [ ]:
waveforms_surface

In [ ]:

surface_channel_session_id=npc_session.SessionRecord('660023_2023-08-08').with_idx(1)
surface_channel_si = npc_ephys.SpikeInterfaceKS25Data(surface_channel_session_id)
probe='C'

###this gets the peak channels of the units
units_surface=npc_ephys.get_amplitudes_waveforms_channels_ks25(surface_channel_si, electrode_group_name=probe)

In [ ]:

# surface_channel_si.quality_metrics_df(probe=probe)
units_surface=surface_channel_si.quality_metrics_df(probe=probe)
units_surface=pd.concat([units_surface,surface_channel_si.template_metrics_df(probe=probe)],axis=1)
units_surface['peak_channel']=npc_ephys.get_amplitudes_waveforms_channels_ks25(surface_channel_si, electrode_group_name=probe).peak_channels
if units_surface['peak_channel'].max()<384:
    units_surface['peak_channel']+=384
units_surface['electrode_group_name']='probe'+probe
units_surface['cluster_id']=np.arange(0,len(units_surface))
# add global unit IDs
units_surface=npc_ephys.add_global_unit_ids(units_surface,surface_channel_session_id)
# units_surface.columns

In [ ]:
# units_surface['peak_channel'].unique()
sessionid='660023_2023-08-08'
si = npc_ephys.SpikeInterfaceKS25Data(sessionid)
device_timing_on_sync = npc_ephys.get_ephys_timing_on_sync(npc_lims.get_h5_sync_from_s3(sessionid), 
                                                            npc_lims.get_recording_dirs_experiment_path_from_s3(sessionid),
                                                            only_devices_including='Probe'+probe)

units=npc_ephys.make_units_table_from_spike_interface_ks25(sessionid, device_timing_on_sync)
units=npc_ephys.add_global_unit_ids(units,sessionid)

In [ ]:
units=pd.concat([units,units_surface],axis=0,ignore_index=True)
units

In [ ]:
fig,ax=plt.subplots(1,1)
ax.hist(units['peak_channel'],bins=range(0,769,8))

In [ ]:
# ephys_sessions[0].is_surface_channels
session_info = ephys_sessions[1]
probe='A'

device_timing_on_sync = npc_ephys.openephys.get_ephys_timing_on_sync(npc_lims.get_h5_sync_from_s3(session_info.id+'_1'), 
                                                                        npc_lims.get_recording_dirs_experiment_path_from_s3(session_info.id+'_1'),
                                                                        only_devices_including='Probe'+probe)

units=npc_ephys.make_units_table_from_spike_interface_ks25(str(session_info.id)+'_1', device_timing_on_sync)

In [7]:
#load pickle file from loadpath
import pickle
loadpath=r"\\allen\programs\mindscope\workgroups\dynamicrouting\Ethan\metrics for alignment\except_dict_backup.pkl"
with open(loadpath, 'rb') as f:
    except_dict = pickle.load(f)

In [8]:
except_dict

{'702136_2024-03-06_0_F': RuntimeError('Error fetching units for 702136_2024-03-06 - see original exception above/below'),
 '702136_2024-03-05': ValueError('Session 702136_2024-03-05 has no sorted data assets'),
 '702131_2024-02-26': ValueError('Session 702131_2024-02-26 has no sorted data assets'),
 '716465_2024-02-14': ValueError('Session 716465_2024-02-14 has no sorted data assets'),
 '676909_2023-12-12_0_A': ValueError('Only one class present in y_true. ROC AUC score is not defined in that case.'),
 '676909_2023-12-12_0_B': ValueError('Only one class present in y_true. ROC AUC score is not defined in that case.'),
 '676909_2023-12-12_0_C': ValueError('Only one class present in y_true. ROC AUC score is not defined in that case.'),
 '676909_2023-12-12_0_D': ValueError('Only one class present in y_true. ROC AUC score is not defined in that case.'),
 '676909_2023-12-12_0_E': ValueError('Only one class present in y_true. ROC AUC score is not defined in that case.'),
 '676909_2023-12-12_